In [202]:
import pandas as pd
import unicodedata
import re
import contractions
import nltk
import string
import spacy

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models


C:\Users\gtanwar2\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
def load_data(file):
    return pd.read_csv(file)

In [31]:
tweets_df=load_data(r'C:\Users\gtanwar2\Elon Musk Tweet Analysis\Elon_Musk_Tweets.csv')

In [32]:
tweets_df.head()

,Unnamed: 0,Tweets
0,0,A collaboration of PC x Elon Musk would be rid...
1,1,"Thanks to Elon Musk and Jeff Bezos, for showin..."
2,2,@elonmusk Yes Elon Musk we know. You are at t...
3,3,@Popehat one of the places with the most in-de...
4,4,@suchetadalal @narendramodi @elonmusk Elon Mus...


In [33]:
tweets_df=tweets_df[['Tweets']]

In [34]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35593 entries, 0 to 35592
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweets  35593 non-null  object
dtypes: object(1)
memory usage: 278.2+ KB


In [35]:
def get_number_of_links(documents):
    print("{:.2f}% of documents contain links".format(sum(documents.apply(lambda x:x.find('http'))>0)/len(documents)*100))
def get_number_of_links(documents):
    print("{:.2f}% of documents contain links".format(sum(documents.apply(lambda x:x.find('http'))>0)/len(documents)*100))

In [36]:
get_number_of_links(tweets_df.Tweets)

57.38% of documents contain links


In [37]:
def to_lowercase(text):
    return text.lower()

In [38]:
tweets_df.Tweets=tweets_df.Tweets.apply(to_lowercase)

In [39]:
def remove_accented_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [40]:
tweets_df.Tweets=tweets_df.Tweets.apply(remove_accented_chars)

In [41]:
def remove_link(text):
    return re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)

In [42]:
tweets_df.Tweets=tweets_df.Tweets.apply(remove_link)

In [43]:
def expand_contractions(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))   
    return ' '.join(expanded_words)

In [44]:
tweets_df.Tweets=tweets_df.Tweets.apply(expand_contractions)

In [47]:
def remove_mentions_and_tags(text):
    text = re.sub(r'@\S*', '', text)
    return re.sub(r'#\S*', '', text)

In [77]:
def keep_only_alphabet(text):
    return re.sub(r'[^a-z]', ' ', text)

In [79]:
tweets_df.Tweets=tweets_df.Tweets.apply(remove_tags)

In [81]:
tweets_df.Tweets=tweets_df.Tweets.apply(keep_only_alphabet)

In [97]:
def remove_stopwords(text,nlp,custom_stop_words=None,remove_small_tokens=True,min_len=2):
    if custom_stop_words:
        nlp.Defaults.stop_words |= custom_stop_words
    filtered_sentence =[] 
    doc=nlp(text)
    for token in doc:
        lexeme = nlp.vocab[token.text]    
        if lexeme.is_stop == False: 
            if remove_small_tokens:
                if len(token.text)>min_len:
                    filtered_sentence.append(token.text)
            else:
                filtered_sentence.append(token.text)
            
    return " ".join(filtered_sentence) if len(filtered_sentence)>0 else None

In [113]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
tweets_df.Tweets=tweets_df.Tweets.apply(lambda x:remove_stopwords(x,nlp,{"elon","musk",}))

In [120]:
tweets_df.dropna(inplace=True)

In [142]:
def lemmatize(text, nlp):
    doc = nlp(text)
    lemmatized_text = []
    for token in doc:
        lemmatized_text.append(token.lemma_)
    return " ".join(lemmatized_text)

In [144]:
tweets_df.Tweets=tweets_df.Tweets.apply(lambda x:lemmatize(x,nlp))

In [148]:
tweets_df.to_csv('processed_tweets.csv')

In [193]:
def generate_words(tweet):
    words=[]
    for word in tweet.split(' '):
        if word!='':
            words.append(word)
    return words

In [196]:
tweets_df['words']=tweets_df.Tweets.apply(generate_words)

In [197]:
tweets_df.head()

,Tweets,words
0,collaboration ridiculous break internet materi...,"[collaboration, ridiculous, break, internet, m..."
1,thank jeff bezo show people billionaire act ha...,"[thank, jeff, bezo, show, people, billionaire,..."
2,yes know,"[yes, know]"
3,place indepth conversation jack dorsey bernie ...,"[place, indepth, conversation, jack, dorsey, b..."
4,move red carpet welcome inherit like tata...,"[move, red, carpet, welcome, inherit, like, ta..."


In [221]:
def create_dictionary(tweets_df):
    return corpora.Dictionary(tweets_df.words)
id2word=create_dictionary(tweets_df) 

In [260]:
print(id2word)

Dictionary(19692 unique tokens: ['break', 'collaboration', 'influential', 'internet', 'material']...)


In [222]:
def create_document_matrix(tokens):
    corpus = []
    for text in tokens:
        new = id2word.doc2bow(text)
        corpus.append(new)
    return corpus

In [237]:
id2word = corpora.Dictionary(tweets_df.words)

corpus = []
for text in tweets_df.words:
    new = id2word.doc2bow(text)
    corpus.append(new)

In [256]:
(set(tweets_df.words[2]))

{'know', 'yes'}

In [248]:
print((corpus[0]))


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]


In [262]:
tweets_df.words[0]

['collaboration',
 'ridiculous',
 'break',
 'internet',
 'material',
 'influential',
 'people',
 'visionarie',
 'unique',
 'personality',
 'people',
 'special']

In [211]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [223]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 10);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [224]:
get_lda_topics(lda_model,10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,moon,good,support,amp,land,start,news,like,forever,floki
1,fly,doge,potential,crypto,think,steal,money,world,people,tesla
2,song,get,tweet,amazing,marketcap,plan,tunnel,need,bitcoin,send
3,broadcast,coin,musk,new,wish,change,worry,help,big,famous
4,stealh,ceo,company,goat,dream,genius,michael,high,year,say
5,baby,instead,twitter,important,look,loop,canadian,want,laser,starlink
6,dance,wallet,car,give,space,call,month,low,go,launch
7,tribute,investor,traffic,monument,nft,chip,business,pay,video,price
8,read,gamestop,drive,project,mars,elite,bet,man,know,satellite
9,message,human,way,egt,true,china,retweet,rich,thing,let


In [216]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis


C:\Users\gtanwar2\AppData\Roaming\Python\Python38\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.209748  0.406436       1        1  18.502541
9      0.170458 -0.446689       2        1  18.427149
7      0.210386  0.444966       3        1  12.564513
6     -0.240884 -0.389376       4        1   9.834339
2     -0.375502  0.162388       5        1   9.330223
1      0.139590  0.224507       6        1   8.042887
4      0.448893 -0.142086       7        1   7.612718
3     -0.398393 -0.112796       8        1   6.590120
0      0.295110  0.005516       9        1   5.188432
5     -0.039909 -0.152867      10        1   3.907078, topic_info=           Term         Freq        Total Category  logprob  loglift
177        moon  4810.000000  4810.000000  Default  30.0000  30.0000
282        good  3632.000000  3632.000000  Default  29.0000  29.0000
2021       land  3068.000000  3068.000000  Default  28.0000  28.0000
118        news  2928.000000  2928.000000  Default  27.0000  27.0000
404         fly  2268.000000  2268.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1602      gates   104.615780   105.463361  Topic10  -4.7642   3.2343
1160      plane    95.676519    96.524827  Topic10  -4.8535   3.2336
2421       iron    93.253913    94.101476  Topic10  -4.8792   3.2333
1600  corporate    92.801144    93.648954  Topic10  -4.8840   3.2333
576   metaverse   102.666944   402.826253  Topic10  -4.7830   1.8754

[335 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1527      3  0.999087  address
78        5  0.994348    agree
1365      3  0.996665  airdrop
614       8  0.999579  amazing
108       8  0.999328      amp
...     ...       ...      ...
1083      4  0.998552    worry
635       8  0.998001    worth
1723      5  0.993451    write
1198      1  0.998387    wrong
663       1  0.999179     year

[319 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 8, 7, 3, 2, 5, 4, 1, 6])